### In this file: 
Lazy Classification for all models.

Here the results are not as bad as I previously thought. Lazy classification for 15 features selected by PPS score achieves around 64% F1 score. It's not bad!


In [2]:
from insignificant.SupervisedOverr import LazyClassifier

from scipy.stats import randint
import pandas as pd
from IPython.core.display_functions import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from helper import *
from feature_selection import *
import graphviz
from IPython.display import Image
from IPython.display import display
from sklearn.tree import export_graphviz
import seaborn as sns
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from helper import *
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Supress all warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
lipidomics, metabolomics, proteomics = load_data()
print("Lipidomics: ", lipidomics.shape)
print("Metabolomics: ", metabolomics.shape)
print("Proteomics: ", proteomics.shape)

filter_data = False
scale_data = True

if filter_data:
    lipidomics = filter_omics(lipidomics, fraction=0.85)
    metabolomics = filter_omics(metabolomics, fraction=0.9)
    proteomics = filter_omics(proteomics, fraction=0.85)
if scale_data:
    metabolomics = pareto_scaling(metabolomics)
    proteomics = pareto_scaling(proteomics)
    
    print("\nLipidomics: ", lipidomics.shape)
    print("Metabolomics: ", metabolomics.shape)
    print("Proteomics: ", proteomics.shape, "\n")

X_train, X_test, Y_train, Y_test = concatenate_data(
    lipidomics, metabolomics, proteomics, resample_train=False, resample_test=False
)

Lipidomics:  (96, 581)
Metabolomics:  (96, 216)
Proteomics:  (96, 323)

Lipidomics:  (96, 581)
Metabolomics:  (96, 215)
Proteomics:  (96, 322) 

Training set shape:  (58, 1116)
Testing set shape:  (38, 1116)


In [4]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(
    X_train, X_test, 
    Y_train, Y_test)

print(models)

100%|██████████| 29/29 [00:03<00:00,  7.95it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001566 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23246
[LightGBM] [Info] Number of data points in the train set: 58, number of used features: 1111
[LightGBM] [Info] Start training from score -0.563935
[LightGBM] [Info] Start training from score -1.575536
[LightGBM] [Info] Start training from score -1.495494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

# Again, on the 15 features using PPS score

In [7]:
lipidomics_features = [
    'LPE_22_6_SN1_', 'PC_16_0_18_3_B_', 'PC_33_0_B_',
    'LPC_18_2_SN2_', 'DHCER_D18_0_20_0_', 'RID', 'ThreeClass']
metabolomics_features = [
    'APOB.APOA1', 'M.LDL.CE_.', 'REMNANT.C',
    'XL.VLDL.FC', 'XS.VLDL.FC', 'RID', 'ThreeClass']
proteomics_features = [
    'EVAFDLEIPK', 'HLSLLTTLSNR', 'LSITGTYDLK',
    'YIVSGTPTFVPYLIK', 'LSILYPATTGR',  'RID', 'ThreeClass']

lipidomics_filtered = lipidomics[lipidomics_features]
metabolomics_filtered = metabolomics[metabolomics_features]
proteomics_filtered = proteomics[proteomics_features]

X_train_filtered, X_test_filtered, Y_train_filtered, Y_test_filtered = concatenate_data(
    lipidomics_filtered, metabolomics_filtered, proteomics_filtered
)

Training set shape:  (58, 19)
Testing set shape:  (38, 19)


In [9]:
clf_filtered = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, _ = clf_filtered.fit(
    X_train_filtered, X_test_filtered, 
    Y_train_filtered, Y_test_filtered
)

print(models)

100%|██████████| 29/29 [00:01<00:00, 26.01it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 311
[LightGBM] [Info] Number of data points in the train set: 58, number of used features: 15
[LightGBM] [Info] Start training from score -0.563935
[LightGBM] [Info] Start training from score -1.575536
[LightGBM] [Info] Start training from score -1.495494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt